In [37]:
# https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [2]:
# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [6]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [7]:
n_chars = len(raw_text)
n_vocab = len(chars)
print "Total Characters: ", n_chars
print "Total Vocab: ", n_vocab

Total Characters:  144395
Total Vocab:  47


In [13]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print "Total Patterns: ", n_patterns



Total Patterns:  144295


In [22]:
## One-hot encoding

# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [31]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [32]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [40]:
model.fit(X, y, epochs=8, batch_size=128, callbacks=callbacks_list)

Epoch 1/8
144295/144295 [==============================] - 1302s 9ms/step - loss: 2.5277

Epoch 00001: loss improved from 2.58828 to 2.52767, saving model to weights-improvement-01-2.5277.hdf5
Epoch 2/8
144295/144295 [==============================] - 1292s 9ms/step - loss: 2.4771

Epoch 00002: loss improved from 2.52767 to 2.47708, saving model to weights-improvement-02-2.4771.hdf5
Epoch 3/8
144295/144295 [==============================] - 1482s 10ms/step - loss: 2.4289

Epoch 00003: loss improved from 2.47708 to 2.42889, saving model to weights-improvement-03-2.4289.hdf5
Epoch 4/8
144295/144295 [==============================] - 1289s 9ms/step - loss: 2.3838

Epoch 00004: loss improved from 2.42889 to 2.38380, saving model to weights-improvement-04-2.3838.hdf5
Epoch 5/8
144295/144295 [==============================] - 1282s 9ms/step - loss: 2.3397

Epoch 00005: loss improved from 2.38380 to 2.33965, saving model to weights-improvement-05-2.3397.hdf5
Epoch 6/8
144295/144295 [=========

In [43]:
# Generating Text with an LSTM Network

# load the network weights
filename = "weights-improvement-08-2.2146.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# reverse mapping

int_to_char = dict((i, c) for i, c in enumerate(chars))

# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print "Seed:"
print "\"", ''.join([int_to_char[value] for value in pattern]), "\""

# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print "\nDone."

Seed:
" u mean by that?' said the caterpillar sternly. 'explain
yourself!'

'i can't explain myself, i'm afr "
aid ' shi gatter seid to the grrphon. 
'ie io wou dien ' said the macc te the drrpoon. 
'ie io wou ain toe teie ' said the macc te the drrpoon. 
'ie io wou ain toe teie ' said the macc te the drrpoon. 
'ie io wou ain toe teie ' said the macc te the drrpoon. 
'ie io wou ain toe teie ' said the macc te the drrpoon. 
'ie io wou ain toe teie ' said the macc te the drrpoon. 
'ie io wou ain toe teie ' said the macc te the drrpoon. 
'ie io wou ain toe teie 

KeyboardInterrupt: 